<a href="https://colab.research.google.com/github/JayChaoNTPUACC/Early-Warning-For-Stocks-With-Abnormal-Trading-Conditions-Based-On-Generative-Adversarial-Networks/blob/main/GetData/getdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#安裝套件
!pip install ffn
!pip install FinMind
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
ext = '0.4.0-oneiric1_amd64.deb -qO'
!wget $url/libta-lib0_$ext libta.deb
!wget $url/ta-lib0-dev_$ext ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

(Reading database ... 120906 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link



In [2]:
#import套件
import os
import pandas as pd
import ffn
import FinMind
import talib
from talib import abstract
import requests
from datetime import datetime,timedelta

In [3]:
#讀取資料
df_punishdata = pd.read_excel("punish.xlsx")
#將證券代號型別轉為字串
df_punishdata["證券代號"] = df_punishdata["證券代號"].astype(str)

In [4]:
df_punishdata

,公布日期,證券代號,證券名稱,處置條件,代碼,處置開始時間,處置結束時間
0,112/03/24,1435,中福,連續三次,1,112/03/27,112/04/12
1,112/06/28,1471,首利,連續三次,1,112/06/29,112/07/12
2,112/06/29,1519,華城,最近三十個營業日已有十二次,6,112/06/30,112/07/13
3,112/04/19,1529,樂事綠能,連續三次,1,112/04/20,112/05/04
4,112/04/28,1727,中華化,連續三次,1,112/05/02,112/05/15
5,112/04/20,1810,和成,連續三次,1,112/04/21,112/05/05
6,112/06/28,2059,川湖,連續三次,1,112/06/29,112/07/12
7,112/07/21,2316,楠梓電,連續三次,1,112/07/24,112/08/04
8,112/07/31,2329,華泰,連續三次,1,112/08/01,112/08/14
9,112/07/19,2356,英業達,連續三次,1,112/07/20,112/08/02


In [17]:
i = 1
for index, row in df_punishdata.iterrows():
  stock_num = row["證券代號"]
  announced_day = str(int(row["公布日期"].split('/')[0])+1911) + "-" + str(row["公布日期"].split('/')[1]) + "-" + str(row["公布日期"].split('/')[2])
  # print(announced_day)
  stock_name = row["證券名稱"]
  stock_punishreason_kind = row["代碼"]
  punish_startdate = str(int(row["處置開始時間"].split('/')[0])+1911) + "-" + str(row["處置開始時間"].split('/')[1]) + "-" + str(row["處置開始時間"].split('/')[2])
  punish_enddate = str(int(row["處置結束時間"].split('/')[0])+1911) + "-" + str(row["處置結束時間"].split('/')[1]) + "-" + str(row["處置結束時間"].split('/')[2])
  url = "https://api.finmindtrade.com/api/v4/data"
  token = ""
  data_startdate = datetime.strftime(datetime.strptime(announced_day, '%Y-%m-%d') - timedelta(days = 182), '%Y-%m-%d')
  data_enddate = announced_day
  # print(data_enddate)
  parameter = {
      "dataset": "TaiwanStockPrice",
      "data_id": stock_num,
      "start_date": data_startdate,
      "end_date": data_enddate,
      "token": token
  }
  get_stock_deal_info = pd.DataFrame(requests.get(url, params=parameter).json()["data"])
  # print(get_stock_deal_info)
  # ta_list = talib.get_functions()
  # ta_list = ['HT_DCPERIOD','HT_DCPHASE','HT_PHASOR','HT_SINE','HT_TRENDMODE','MINMAXINDEX','MACD','RSI','BBANDS','CCI','DX','ADX','STOCH']
  # ta_list = ['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE', 'DIV', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'SUB', 'SUM', 'ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH', 'ADXR', 'APO', 'AROONOSC', 'CCI', 'CMO', 'MACD', 'MACDEXT', 'MACDFIX', 'MINUS_DI', 'MOM', 'PLUS_DM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCHF', 'STOCHRSI', 'TRIX', 'WILLR', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MIDPOINT', 'SAR', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'CDL3BLACKCROWS', 'CDL3LINESTRIKE', 'CDL3STARSINSOUTH', 'CDLABANDONEDBABY', 'CDLBELTHOLD', 'CDLCLOSINGMARUBOZU', 'CDLCOUNTERATTACK', 'CDLDOJI', 'CDLDRAGONFLYDOJI', 'CDLEVENINGDOJISTAR', 'CDLGAPSIDESIDEWHITE', 'CDLHAMMER', 'CDLHARAMI', 'CDLHIGHWAVE', 'CDLHIKKAKEMOD', 'CDLIDENTICAL3CROWS', 'CDLINVERTEDHAMMER', 'CDLKICKINGBYLENGTH', 'CDLLONGLEGGEDDOJI', 'CDLMARUBOZU', 'CDLMATHOLD', 'CDLMORNINGSTAR', 'CDLPIERCING', 'CDLRISEFALL3METHODS', 'CDLSHOOTINGSTAR', 'CDLSPINNINGTOP', 'CDLSTICKSANDWICH', 'CDLTASUKIGAP', 'CDLTRISTAR', 'CDLUPSIDEGAP2CROWS', 'AVGPRICE', 'TYPPRICE', 'BETA', 'LINEARREG', 'LINEARREG_ANGLE', 'LINEARREG_INTERCEPT', 'LINEARREG_SLOPE', 'STDDEV', 'TSF', 'VAR', 'NATR', 'AD', 'OBV']
  ta_list = ['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'SUM', 'ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH', 'APO', 'CMO', 'MACD', 'MACDEXT', 'MACDFIX', 'MOM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCHRSI', 'TRIX', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MIDPOINT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'LINEARREG', 'LINEARREG_ANGLE', 'LINEARREG_INTERCEPT', 'LINEARREG_SLOPE', 'STDDEV', 'TSF', 'VAR']
  for x in ta_list:
      try:
          output = eval('abstract.'+x+'(get_stock_deal_info)')
          #print(output)
          #print(type(output))
          output.name = x.lower() if type(output) == pd.core.series.Series else None
          #print(get_stock_deal_info.tail(60))
          #print(pd.DataFrame(output).tail(60))
          get_stock_deal_info = pd.concat([get_stock_deal_info, pd.DataFrame(output)], axis=1)
          #print(get_stock_deal_info)
      except:
          print(x)
          ta_list.remove(x)
  print(ta_list)
  if i == 1:
    df_alldatas = get_stock_deal_info
  else:
    df_alldatas = pd.concat([df_alldatas, get_stock_deal_info], ignore_index=True)
  i += 1
df_alldatas


['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'SUM', 'ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH', 'APO', 'CMO', 'MACD', 'MACDEXT', 'MACDFIX', 'MOM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCHRSI', 'TRIX', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MIDPOINT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'LINEARREG', 'LINEARREG_ANGLE', 'LINEARREG_INTERCEPT', 'LINEARREG_SLOPE', 'STDDEV', 'TSF', 'VAR']
['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'SUM', 'ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH', 'APO', 'CMO', 'MACD', 'MACDEXT', 'MACDFIX', 'MOM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCHRSI', 'TRIX', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE',

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover,...,tema,trima,wma,linearreg,linearreg_angle,linearreg_intercept,linearreg_slope,stddev,tsf,var
0,2023-03-20,1435,67302,2700989,40.10,44.20,39.80,39.80,0.00,169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-03-21,1435,55650,2105853,35.90,40.10,35.85,38.00,-1.80,139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-03-22,1435,161692,5674857,37.25,37.25,34.20,35.35,-2.65,222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-03-23,1435,42130,1427894,35.00,35.00,33.20,34.15,-1.20,105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-03-24,1435,44945,1473605,32.60,33.35,32.25,32.25,-1.90,117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.694698,NaN,7.2614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5782,2023-02-22,8996,15275215,3666810981,238.00,246.50,227.50,242.00,3.00,10027,...,243.448004,209.210417,221.258065,243.514286,64.372742,216.414286,2.084615,3.322650,245.598901,11.0400
5783,2023-02-23,8996,13890045,3293631596,242.00,245.00,233.00,234.00,-8.00,9645,...,243.728766,211.691667,222.824731,242.057143,58.379522,220.942857,1.624176,3.429286,243.681319,11.7600
5784,2023-02-24,8996,10135285,2415411520,238.50,240.50,234.50,239.00,5.00,6194,...,244.765508,214.175000,224.615054,241.985714,54.010710,224.085714,1.376923,2.756810,243.362637,7.6000
5785,2023-03-01,8996,12219884,2994677556,239.50,250.50,238.50,244.50,5.50,9066,...,246.557625,216.691667,226.662366,242.642857,49.619923,227.357143,1.175824,3.515679,243.818681,12.3600


In [6]:
df_alldatas.count()

date                   5787
stock_id               5787
Trading_Volume         5787
Trading_money          5787
open                   5787
                       ... 
linearreg_intercept    5145
linearreg_slope        5145
stddev                 5587
tsf                    5145
var                    5587
Length: 85, dtype: int64

In [18]:
df_alldatas = pd.merge(df_alldatas,df_punishdata, left_on='stock_id', right_on='證券代號')

In [8]:
df_alldatas

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover,...,stddev,tsf,var,公布日期,證券代號,證券名稱,處置條件,代碼,處置開始時間,處置結束時間
0,2023-03-20,1435,67302,2700989,40.10,44.20,39.80,39.80,0.00,169,...,NaN,NaN,NaN,112/03/24,1435,中福,連續三次,1,112/03/27,112/04/12
1,2023-03-21,1435,55650,2105853,35.90,40.10,35.85,38.00,-1.80,139,...,NaN,NaN,NaN,112/03/24,1435,中福,連續三次,1,112/03/27,112/04/12
2,2023-03-22,1435,161692,5674857,37.25,37.25,34.20,35.35,-2.65,222,...,NaN,NaN,NaN,112/03/24,1435,中福,連續三次,1,112/03/27,112/04/12
3,2023-03-23,1435,42130,1427894,35.00,35.00,33.20,34.15,-1.20,105,...,NaN,NaN,NaN,112/03/24,1435,中福,連續三次,1,112/03/27,112/04/12
4,2023-03-24,1435,44945,1473605,32.60,33.35,32.25,32.25,-1.90,117,...,2.694698,NaN,7.2614,112/03/24,1435,中福,連續三次,1,112/03/27,112/04/12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6959,2023-02-24,8996,10135285,2415411520,238.50,240.50,234.50,239.00,5.00,6194,...,2.756810,243.362637,7.6000,112/03/02,8996,高力,連續五次及當日沖銷標準,4,112/03/03,112/03/20
6960,2023-03-01,8996,12219884,2994677556,239.50,250.50,238.50,244.50,5.50,9066,...,3.515679,243.818681,12.3600,112/06/07,8996,高力,連續五次,3,112/06/08,112/06/21
6961,2023-03-01,8996,12219884,2994677556,239.50,250.50,238.50,244.50,5.50,9066,...,3.515679,243.818681,12.3600,112/03/02,8996,高力,連續五次及當日沖銷標準,4,112/03/03,112/03/20
6962,2023-03-02,8996,5355469,1308148387,245.50,248.00,241.00,241.00,-3.50,4091,...,3.527038,242.719780,12.4400,112/06/07,8996,高力,連續五次,3,112/06/08,112/06/21


In [19]:
output_file_name = 'get_stocks_result.csv'
df_alldatas.to_csv(output_file_name,sep=',',index=False,header=True)